In [3]:
import pandas as pd
import numpy as np

# Read the file
file_path = 'Datasets/1_Geldermalsen.txt'

# Read the general parameters line
with open(file_path, 'r') as file:
    general_params = file.readline().strip()

general_params = general_params.replace(" ", "")

horizontal_streets = int(general_params[0])
vertical_street = int(general_params[1])
no_sledges = int(general_params[2])
no_presents = int(general_params[3])
bonus_per_ride = int(general_params[4])
no_steps = int(general_params[5]) 

# Read the rest of the file into a DataFrame
df = pd.read_csv(file_path, sep=' ', skiprows=0, header=None)
df.columns = ['Start_Horizontal', 'Start_Vertical', 'Finish_Horizontal', 'Finish_Vertical', 'Earliest_Start', 'Latest_Finish']

# Displaying the DataFrame
print("General Parameters:", general_params)
print("Delivery Data:")
df

#sort on starting time
df.sort_values(by=["Earliest_Start","Latest_Finish"])



class present:
    def __init__(self, a,b,x,y,s,f,v,nr):
        self.starthor = a
        self.startver = b
        self.finhor = x
        self.finver = y
        self.devtime = s
        self.fintime = f
        self.visited = v
        self.number = nr
        

presentarray = np.empty(no_presents,dtype = present)
for i in range (no_presents):
    presentarray[i] = present(df.iloc[i][0],df.iloc[i][1],df.iloc[i][2],df.iloc[i][3],df.iloc[i][4],df.iloc[i][5],0,i)

    

#keeps track of route per sledge
class sledge:
    def __init__(self):
        self.no_pres_deliv = 0
        self.no_steps_taken = 0
        self.pres_deliv = [present]
        self.current_hor = 0
        self.current_ver = 0

    #adds a present to the end  of sledge
    def add_present(nr):
        p = presentarray[nr]
        self.no_pres_deliv+=1
        self.no_steps_taken += (abs(self.current_hor-p.starthor)+abs(self.current_ver-p.startver)+abs(p.starthor-p.finhor)+abs(p.startver-p.finver))
        self.pres_deliv.add[p]
        self.current_hor = p.finhor
        self.current_ver = p.finver

    #remove present from the sledge
    def remove_present(indexinpreslist):
        if self.no_pres_deliv==0:#no item to remove
            return
        p=pres_deliv[indexinpreslist]
        if indexinpreslist==0 and indexinpreslist==no_pres_deliv - 1:
            self.no_pres_deliv = 0
            self.no_steps_taken = 0
            self.pres_deliv.Pop()
            self.current_hor=0
            self.current_ver=0
        elif (indexinpreslist==0):
            nextpres = pres_deliv[indexinpreslist+1]
            self.no_pres_deliv-=1
            self.no_steps_taken -= (abs(p.finhor-p.starthor)+abs(p.finver-p.startver)+p.starthor+p.startver+abs(p.finhor-nextpres.starthor)+abs(p.finver-nextpres.startver))
            self.no_steps_taken += (nextpres.starthor + nextpres.startver)
            self.pres_deliv.pop(indexinpreslist)
        elif (indexinpreslist==no_pres_deliv - 1):
            prevpres = pres_deliv[indexinpreslist-1]
            self.no_pres_deliv-=1
            self.no_steps_taken -= (abs(p.finhor-p.starthor)+abs(p.finver-p.startver)+abs(p.starthor-prevpres.finhor)+abs(p.startver-prevpres.finver))
            self.pres_deliv.pop(indexinpreslist)
        else:
            nextpres = pres_deliv[indexinpreslist+1]
            prevpres = pres_deliv[indexinpreslist-1]
            self.no_pres_deliv-=1
            self.no_steps_taken -= (abs(p.finhor-p.starthor)+abs(p.finver-p.startver)+abs(p.starthor-prevpres.finhor)+abs(p.startver-prevpres.finver)+abs(p.finhor-nextpres.starthor)+abs(p.finver-nextpres.startver))
            self.no_steps_taken += (abs(nextpres.starthor-prevpres.finhor) + abs(nextpres.startver-prevpres.finver))
            self.pres_deliv.pop(indexinpreslist)





        

#gives route
class route:
    array_sledges = np.empty(no_sledges, dtype=sledge)
    for i in range(0,no_sledges):
        array_sledges[i] = sledge()
    
    
        






General Parameters: 3423210
Delivery Data:
